# Pandas Data Generation, Database Creation and Dump

We will use Faker, a python library for synthetic data generation to create the Customers, Logins and f_amount tables and finally dump to a single database using SQLite3.  If you want to install faker do a pip install faker uncommenting the following row

In [1]:
#pip install faker

### I - Create the dataframes

In [2]:
from faker import Faker
from itertools import repeat
import pandas as pd
import numpy as np
import random
fake = Faker()

#### Creation of the Customers Table

The customers table has the cid (client id), firt name, last name, date and email columns

In [3]:
data = []
for cid in random.sample(range(10000), 1000):
    first_name, last_name = fake.first_name(), fake.last_name()
    dotcom = fake.ascii_company_email().split('@')[1]
    email = f'{first_name}.{last_name}@{dotcom}'
    date = fake.date()
    data.append([cid, date,first_name, last_name, email])

In [4]:
customers = pd.DataFrame(data, columns=['cid', 'date', 'first_name', 'last_name', 'email'])
customers.head()

,cid,date,first_name,last_name,email
0,127,1986-03-15,Erik,Foster,Erik.Foster@abbott.com
1,6669,1979-04-22,Robert,Hall,Robert.Hall@doyle.com
2,405,2008-04-29,Melissa,Manning,Melissa.Manning@chung.biz
3,3745,1996-09-20,Christine,Reed,Christine.Reed@nelson.com
4,6436,1991-03-20,Brittany,Hurley,Brittany.Hurley@oliver-turner.biz


#### Creation of the Logins Table

The logins table has the cid (client id), and date columns. The date represents the last date of login

In [5]:
data = []
index = []
for _ in range(10000):
    login_date = fake.date()
    cid = random.randint(1, 10000)
    data.append([cid, login_date])

In [6]:
logins = pd.DataFrame(data, columns=['cid', 'date'])
logins.head()

,cid,date
0,9274,1992-01-30
1,5075,2009-11-14
2,13,1988-06-19
3,930,1973-05-02
4,1438,1983-05-03


#### Creation of the f_amount Table

The logins table has the cid (client id), date of the transaction, amount of the transaction and the source company of transaction

In [7]:
data = []
for cid in random.sample(range(10000), 5000):
    date = fake.date()
    amount = np.random.random()*random.randint(40, 100000)
    source = fake.company()
    data.append([cid, date, np.round(amount, 2), source])

In [8]:
f_amount = pd.DataFrame(data, columns=['cid', 'date', 'amount', 'source'])
f_amount.head()

,cid,date,amount,source
0,3057,1972-07-06,16054.20,Gonzalez LLC
1,2744,1999-01-29,48105.86,"Carter, Klein and Jackson"
2,4230,1991-03-03,50290.17,Willis-Perez
3,2072,1991-03-20,9993.05,"Nelson, Thomas and Townsend"
4,3599,2016-05-10,2894.98,Oconnor Group


### II - Creation of the sqlite3 database

Below we will create a sqlite3 database and dump all the values into the tables.

#### Create an sqlite3 table

In [9]:
import sqlite3

connection = sqlite3.connect('excercise.db')

In [10]:
cursor = connection.cursor()

#### create 3 tables

In [11]:
cursor.execute('''CREATE TABLE IF NOT EXISTS Customers
              (cid INT, date DATE, first_name TEXT, last_name TEXT, email TEXT )''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Logins (cid INT, date DATE)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS f_amount (cid INT, date DATE, amount FLOAT, source TEXT)''')

#### Dump the pandas dataframes to the respective tables

In [12]:
for _, values in customers.iterrows():
        sql_statement = '''INSERT INTO Customers (cid, date, first_name, last_name, email)  VALUES(?, ?, ?, ?, ?)''' 
        data_tuple = tuple(values)
        cursor.execute(sql_statement, data_tuple)
        
for _, values in logins.iterrows():
        sql_statement = '''INSERT INTO Logins (cid, date)  VALUES(?, ?)'''
        data_tuple = tuple(values)
        cursor.execute(sql_statement, data_tuple)
        
        
for _, values in f_amount.iterrows():
        sql_statement = '''INSERT INTO f_amount (cid, date, amount, source) VALUES(?, ?, ?, ?)'''
        data_tuple = tuple(values)
        cursor.execute(sql_statement, data_tuple)

#### Commit to save changes and close connection

In [13]:
connection.commit()
connection.close()